<a href="https://colab.research.google.com/github/as9786/ML-DLPratice/blob/main/%EC%98%A4%EC%B0%A8%EC%97%AD%EC%A0%84%ED%8C%8C2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 역전파 학습법을 이용한 심층 신경망 학습

In [1]:
import time
import numpy as np

# Utility function

In [2]:
def _t(x):
  return np.transpose(x)

def _m(A, B):
  return np.matmul(A, B)

# Sigmoid

In [3]:
class Sigmoid:
  def __init__(self):
    self.last_o = 1 # 마지막 출력 저장. 1로 해주는 이유는 곱해도 값이 변하지 않음
  
  def __call__(self,x):
    self.last_o = 1 / (1.0 + np.exp(-x))
    return self.last_o

  def grad(self): # sigmoid(x)(1-sigmoid(x))
    return self.last_o * (1 - self.last_o)

# MSE 구현

In [4]:
class MeanSquaredError:
  def __init__(self):
    # 경사 저장
    self.dh = 1
    self.last_diff = 1

  def __call__(self,h,y): # 1/2 * mean((h - y)^2)
    self.last_diff = h - y
    return 1/2 * np.mean(np.square(h-y))

  def grad(self): # h - y
    return self.last_diff

# 신경망 구현

In [19]:
class Neuron:
  def __init__(self,W,b,a_obj):
    # 매개변수
    self.W = W
    self.b = b
    self.a = a_obj()

    # 경사 
    self.dW = np.zeros_like(self.W)
    self.db = np.zeros_like(self.b)
    self.dh = np.zeros_like(_t(self.W))

    self.last_x = np.zeros((self.W.shape[0]))
    self.last_h = np.zeros((self.W.shape[1]))
  
  def __call__(self,x):
    self.last_x = x
    self.last_h = _m(_t(self.W),x) + self.b
    return self.a(self.last_h)

  def grad(self): # dy/dh = W
    return self.W * self.a.grad()

  def grad_W(self,dh):
    grad = np.ones_like(self.W)
    grad_a = self.a.grad()
    for j in range(grad.shape[1]): # y = w*Tx + b, dy/dw = x
      grad[:,j] = dh[j] + grad_a[j] * self.last_x
    return grad

  def grad_b(self,dh): # dy/dh = 1
    return dh * self.a.grad() * 1

# 심층 신경망 구현

In [23]:
class DNN:
  def __init__(self,hidden_depth,num_neuron,num_input,num_output,activation=Sigmoid):
    def init_var(i,o):
      return np.random.normal(0.0,0.01,(i,o)),np.zeros((o,))

    self.sequence = list()
    # 첫 은닉층
    W,b = init_var(num_input,num_neuron)
    self.sequence.append(Neuron(W,b,activation))

    # 은닉층들
    for _ in range(hidden_depth):
      W,b = init_var(num_neuron,num_neuron)
      self.sequence.append(Neuron(W,b,activation))
    
    # 출력층
    W,b = init_var(num_neuron,num_output)
    self.sequence.append(Neuron(W,b,activation))
  
  def __call__(self,x):
    for layer in self.sequence:
      x = layer(x)
    return x 

  def calc_gradient(self,loss_obj):
    loss_obj.dh = loss_obj.grad()
    self.sequence.append(loss_obj)

    # 역전파
    for i in range(len(self.sequence)-1,0,-1):
      l1 = self.sequence[i]
      l0 = self.sequence[i-1] 

      l0.dh = _m(l0.grad(),l1.dh)
      l0.dW = l0.grad_W(l1.dh)
      l0_db = l0.grad_b(l1.dh)
      
    self.sequence.remove(loss_obj)

# 경사하강 학습

In [26]:
def gradient_descent(network,x,y,loss_obj,alpha=0.01):
  loss = loss_obj(network(x),y)
  network.calc_gradient(loss_obj)
  for layer in network.sequence:
    layer.W += -alpha * layer.dW
    layer.b += -alpha * layer.db
  return loss

# 동작

In [27]:
x = np.random.normal(0.0,1.0,(10,))
y = np.random.normal(0.0,1.0,(2,))

dnn = DNN(hidden_depth=5,num_neuron=32,num_input=10,num_output=2,activation=Sigmoid)

t = time.time()
loss_obj = MeanSquaredError()
for epoch in range(100):
  loss = gradient_descent(dnn,x,y,loss_obj,0.01)
  print('Epoch : {} Test loss : {}'.format(epoch,loss))

print('{} seconds elapsed'.format(time.time()-t))

Epoch : 0 Test loss : 0.553041353570619
Epoch : 1 Test loss : 0.5148961570724307
Epoch : 2 Test loss : 0.4806355393274166
Epoch : 3 Test loss : 0.4500353221625107
Epoch : 4 Test loss : 0.42279893342894614
Epoch : 5 Test loss : 0.3986022929360732
Epoch : 6 Test loss : 0.37712191799174166
Epoch : 7 Test loss : 0.35805098105461847
Epoch : 8 Test loss : 0.3411073914604477
Epoch : 9 Test loss : 0.32603691226589687
Epoch : 10 Test loss : 0.3126133594105593
Epoch : 11 Test loss : 0.3006372015125231
Epoch : 12 Test loss : 0.289933375366072
Epoch : 13 Test loss : 0.2803488027853616
Epoch : 14 Test loss : 0.2717498862081238
Epoch : 15 Test loss : 0.2640201322548207
Epoch : 16 Test loss : 0.2570579753284309
Epoch : 17 Test loss : 0.2507748281744773
Epoch : 18 Test loss : 0.24509336080071772
Epoch : 19 Test loss : 0.23994599554150717
Epoch : 20 Test loss : 0.23527359953456234
Epoch : 21 Test loss : 0.23102435349787875
Epoch : 22 Test loss : 0.22715277564557668
Epoch : 23 Test loss : 0.223618880772